In [1]:
import pandas as pd

from datetime import timedelta

In [2]:
data = pd.read_csv('Atraccion_depredadores_PRUEBAS2.csv', encoding='latin-1', delimiter=';')

In [3]:
# data['HORA'].unique()
data['HORA'] = data['HORA'].apply(lambda x: x.replace('.\xa0', '').replace('.', '').strip())


In [4]:
data = data.reset_index()
data.columns = [col.strip() for col in data.columns]

In [5]:
data_cols_a_desnormalizar = data[['index', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11']]

In [6]:
# Identificando las columnas que comienzan con 'T'
t_columns = ['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11']

# Creando un DataFrame "largo" donde cada 'T' es una fila
melted_data = data_cols_a_desnormalizar.melt(id_vars=['index'], value_vars=t_columns, 
                        var_name='Tipo', value_name='Valor')






In [7]:
data_col_orig_faltantes = data[['index', 'SITIO', 'FECHA', 'HORA', 'playlist']]

In [8]:
data_desnormalizada = melted_data.merge(data_col_orig_faltantes, how = 'left', on = ['index'])

In [9]:
# Convertir la columna 'HORA' a datetime y ajustar según 'Tipo'
data_desnormalizada['HORA'] = pd.to_datetime(data_desnormalizada['HORA'], format='%I:%M:%S %p', errors='coerce')




In [10]:
data_desnormalizada['min_a_sumar'] = data_desnormalizada['Tipo'].apply(lambda x: int(x[1:]))

In [11]:
data_desnormalizada['HORA'] = data_desnormalizada['HORA'] + pd.to_timedelta(data_desnormalizada['min_a_sumar'] - 1, unit='m')


In [12]:
# Formateando la columna 'HORA' para mostrar solo hora, minutos y segundos
data_desnormalizada['HORA'] = data_desnormalizada['HORA'].dt.strftime('%H:%M:%S')

In [13]:
# Eliminando la columna auxiliar de minutos añadidos
data_desnormalizada.drop(columns='min_a_sumar', inplace=True)

In [14]:
data_response = data_desnormalizada[['index', 'SITIO', 'FECHA', 'HORA', 'playlist', 'Tipo', 'Valor']]

In [15]:
# Ordenando primero por 'playlist' y luego por 'Tipo'
# Asegurarse que 'playlist' y 'Tipo' están en el formato correcto para ordenamiento
data_response['playlist'] = data_response['playlist'].astype(int)  # Convertir 'playlist' a entero si no lo es
data_response['Tipo_index'] = data_response['Tipo'].apply(lambda x: int(x[1:]))  # Extraer el número del tipo para ordenamiento

# Ordenar primero por 'playlist', luego por 'Tipo_index'
data_response.sort_values(by=['playlist', 'HORA'], inplace=True)

# Eliminar la columna auxiliar 'Tipo_index'
data_response.drop('Tipo_index', axis=1, inplace=True)

In [17]:
detecciones = pd.read_csv('detecciones.csv', encoding='latin-1', delimiter=';')

In [37]:
detecciones['hora_key'] = detecciones['hora'].apply(lambda x: x[:-2] + '00')
                          

In [56]:
completo = data_response.merge(detecciones, how='left', left_on=['FECHA', 'HORA'], right_on=['fecha', 'hora_key'])

In [55]:
def corregir_trat_desfase(row):
    if pd.notnull(row['tratamiento']):
        row['Valor'] = row['tratamiento']
    return row

In [57]:
completo = completo.apply(corregir_trat_desfase, axis=1)

In [59]:
completo['comportamiento'] = completo['comportamiento'].fillna('AUSENCIA')

In [62]:
col_int = ['index', 'SITIO', 'FECHA', 'HORA', 'playlist', 'Tipo', 'Valor', 
       'hora', 'comportamiento', 'id', 'archivo video',
       'archivo tabla de selección', 'observaciones']
completo_col_interes = completo[col_int]

completo_col_interes.columns = ['index', 'sitio', 'fecha', 'hora', 'playlist', 'tipo', 'tratamiento', 
       'hora_especifica', 'comportamiento', 'id', 'archivo_video',
       'archivo_tabla_de_selección', 'observaciones']

In [63]:
completo_col_interes.to_csv('objetivo_1_completo.csv', sep =';', index=False)